In [ ]:
import pandas as pd
import shutil
import os
import numpy as np
import matplotlib.pyplot as plt
import onekey_algo.custom.components as okcomp
from onekey_algo import get_param_in_cwd

plt.rcParams['figure.dpi'] = 300
model_names = ['Clinic', 'Habitat', 'Pathology', 'Ensemble_CH', 'Ensemble_CP', 'Ensemble_HP', 'Ensemble_CHP', 'MHA_CHP']
# 获取配置
task = get_param_in_cwd('task_column') or 'label'
bst_model = get_param_in_cwd('sel_model') or 'LR'
labelf = get_param_in_cwd('label_file') or os.path.join(mydir, 'label.csv')
group_info = get_param_in_cwd('dataset_column') or 'group'

# 读取label文件。
labels = [task]
label_data_ = pd.read_csv(labelf)
label_data_['ID'] = label_data_['ID'].map(lambda x: f"{x}.nii.gz" if not (f"{x}".endswith('.nii.gz') or  f"{x}".endswith('.nii')) else x)
label_data_ = label_data_[['ID', group_info, task]]
label_data_ = label_data_.dropna(axis=0)

ids = label_data_['ID']
print(label_data_.columns)
label_data = label_data_[['ID'] + labels]
label_data

# 训练集-Nomogram

In [ ]:
import pandas as pd
from onekey_algo.custom.components.comp1 import normalize_df

subset = 'train'
Clinic_results = pd.merge(pd.read_csv(f'./results/Clinical_RandomForest_{subset}.csv', header=0), label_data, on='ID', how='inner')
Habitat_results = pd.merge(pd.read_csv(f'./results/Habitat_RandomForest_{subset}.csv', header=0), label_data, on='ID', how='inner')
Pathology_results = pd.merge(pd.read_csv(f'./results/Path_RandomForest_{subset}.csv', header=0), label_data, on='ID', how='inner')
Transformer_results = pd.merge(pd.read_csv(f'./results/Fusion_Transformer_{subset}.csv', header=0), label_data, on='ID', how='inner')

ALL_results = pd.merge(pd.merge(Clinic_results, Habitat_results, on='ID', how='inner'),  Pathology_results, on='ID', how='inner')
ALL_results = pd.merge(ALL_results, Transformer_results, on='ID', how='inner')

ALL_results.columns = ['ID', '-0', model_names[0], task, 
                       '-00', model_names[1], '-l', 
                       '-000', model_names[2], '-ll',
                       '-0000', model_names[-1], '-lll']
ALL_results = normalize_df(ALL_results, not_norm=['ID'], method='minmax')
Clinic = pd.read_csv('data/clinic_mul.csv')
# Clinic['ID'] = Clinic['ID'].map(lambda x: f"{x}.nii.gz")
Clinic = Clinic[[c for c in Clinic.columns if c not in ['label', 'group']]]
ALL_results = pd.merge(ALL_results, Clinic, on='ID', how='inner')
ALL_results = ALL_results.dropna(axis=1)
ALL_results

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from onekey_algo.custom.components import metrics
from sklearn.ensemble import RandomForestClassifier

models = {}
for mn, columns in zip(model_names[-5:-1], [list(Clinic.columns[1:]) + ['Habitat'],
                                      list(Clinic.columns[1:]) + ['Pathology'], 
                                      ['Habitat', 'Pathology'],
                                      list(Clinic.columns[1:]) + ['Habitat', 'Pathology']]):
    model = LogisticRegression(penalty='l2', random_state=0, max_iter=100)
#     model = SVC(probability=True, random_state=0)
#     model = RandomForestClassifier(n_estimators=1, max_depth=None, min_samples_split=2, random_state=0)
#     data_x = ALL_results[list(Clinic.columns[1:]) + model_names[:-1]]
    data_x = ALL_results[columns]
    data_y = ALL_results[task]
    model.fit(data_x, data_y)
    results = model.predict_proba(data_x)
    results = pd.DataFrame(results, index=ALL_results['ID'], columns=[f'-0', f'{mn}']).reset_index()
    results.to_csv(f'./results/{mn}_{subset}.csv', index=False, header=True)
    ALL_results = pd.merge(ALL_results, results, on='ID', how='inner')
    models[mn] = model
ALL_results

In [ ]:
gt = [np.array(ALL_results[task]) for d in model_names]
pred_train = [np.array(ALL_results[d]) for d in model_names]
okcomp.comp1.draw_roc(gt, pred_train, labels=model_names, title=f"Model AUC")
plt.savefig(f'img/{subset}_auc.svg')

In [ ]:
from onekey_algo.custom.components.metrics import analysis_pred_binary
metric = []
youden = {}
for mname, y, score in zip(model_names, gt, pred_train):
    # 计算验证集指标
    acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = analysis_pred_binary(y, score)
    ci = f"{ci[0]:.4f} - {ci[1]:.4f}"
    metric.append((mname, acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres, subset))
    youden[mname] = thres
pd.DataFrame(metric, index=None, columns=['Signature', 'Accuracy', 'AUC', '95% CI', 'Sensitivity', 'Specificity', 
                                          'PPV', 'NPV', 'Precision', 'Recall', 'F1','Threshold', 'Cohort'])

In [ ]:
from onekey_algo.custom.components.delong import delong_roc_test
from onekey_algo.custom.components.comp1 import draw_matrix

delong = []
delong_columns = []
this_delong = []
plt.figure(figsize=(10, 8))
cm = np.zeros((len(model_names), len(model_names)))
for i, mni in enumerate(model_names):
    for j, mnj in enumerate(model_names):
        if i <= j:
            cm[i][j] = np.nan
        else:
            cm[i][j] = delong_roc_test(ALL_results[task], ALL_results[mni], ALL_results[mnj])[0][0]
cm = pd.DataFrame(cm[1:, :-1], index=model_names[1:], columns=model_names[:-1])
draw_matrix(cm, annot=True, cmap='jet_r', cbar=True)
plt.title(f'Cohort {subset} Delong')
plt.savefig(f'img/delong_each_cohort_{subset}.svg', bbox_inches = 'tight')
plt.show()

In [ ]:
from onekey_algo.custom.components.delong import delong_roc_test
from onekey_algo.custom.components.metrics import NRI, IDI

delong = []
delong_columns = []
this_delong = []
plt.figure(figsize=(10, 8))
cm = np.zeros((len(model_names), len(model_names)))
for i, mni in enumerate(model_names):
    for j, mnj in enumerate(model_names):
        cm[i][j] = NRI(ALL_results[mni] > youden[mni], ALL_results[mnj] > youden[mnj], ALL_results[task])
cm = pd.DataFrame(cm, index=model_names, columns=model_names)
draw_matrix(cm, annot=True, cmap='jet_r', cbar=True)
plt.title(f'Cohort {subset} NRI')
plt.savefig(f'img/all_NRI_each_cohort_{subset}.svg', bbox_inches = 'tight')
plt.show()

In [ ]:
from onekey_algo.custom.components.delong import delong_roc_test
from onekey_algo.custom.components.metrics import NRI, IDI

delong = []
delong_columns = []
this_delong = []
cm = np.zeros((len(model_names), len(model_names)))
p = np.zeros((len(model_names), len(model_names)))
for i, mni in enumerate(model_names):
    for j, mnj in enumerate(model_names):
        cm[i][j], p[i][j] = IDI(ALL_results[mni], ALL_results[mnj], ALL_results[task], with_p=True)

for d, n in zip([cm, p], ['IDI', 'IDI pvalue']):
    plt.figure(figsize=(10, 8))
    d = pd.DataFrame(d, index=model_names, columns=model_names)
    draw_matrix(d, annot=True, cmap='jet_r', cbar=True)
    plt.title(f'Cohort {subset} {n}')
    plt.savefig(f'img/all_{n}_each_cohort_{subset}.svg', bbox_inches = 'tight')
    plt.show()

In [ ]:
from onekey_algo.custom.components.comp1 import plot_DCA
plot_DCA([ALL_results[model_name] for model_name in model_names], 
         ALL_results[task], title=f'Model for DCA', labels=model_names, remap=True)
plt.savefig(f'img/{subset}_dca.svg')

In [ ]:
from onekey_algo.custom.components.comp1 import draw_calibration
draw_calibration(pred_scores=pred_train, n_bins=5, remap=True, add_1=True,
                 y_test=gt, model_names=model_names)
plt.savefig(f'img/{subset}_cali.svg')

In [ ]:
from onekey_algo.custom.components import stats

hosmer = []
hosmer.append([stats.hosmer_lemeshow_test(y_true, y_pred, bins=100) 
              for fn, y_true, y_pred in zip(model_names, gt, pred_train)])
pd.DataFrame(hosmer, columns=model_names)

# 绘制Nomogram

In [ ]:
from onekey_algo.custom.components import nomogram
import shutil

ALL_results = ALL_results.round(decimals=2)
ALL_results.columns = [c.replace('+', '_') for c in ALL_results]
nomogram.risk_nomogram(ALL_results, result=task, 
                       columns=list(Clinic.columns[1:]) + ['Habitat'], width=10000, height=4000,
                      x_range='0.05,0.25,0.5,0.75,0.95')

# 测试集-Nomogram

In [ ]:
import pandas as pd
import venn
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from onekey_algo.custom.components import metrics
from sklearn.ensemble import RandomForestClassifier

subset = 'test'
for subset in [s for s in get_param_in_cwd('subsets') if s != 'train']:
    Clinic_results = pd.merge(pd.read_csv(f'./results/Clinical_RandomForest_{subset}.csv', header=0), label_data, on='ID', how='inner')
    Habitat_results = pd.merge(pd.read_csv(f'./results/Habitat_RandomForest_{subset}.csv', header=0), label_data, on='ID', how='inner')
    Pathology_results = pd.merge(pd.read_csv(f'./results/Path_RandomForest_{subset}.csv', header=0), label_data, on='ID', how='inner')
    Transformer_results = pd.merge(pd.read_csv(f'./results/Fusion_Transformer_{subset}.csv', header=0), label_data, on='ID', how='inner')

    ALL_results = pd.merge(pd.merge(Clinic_results, Habitat_results, on='ID', how='inner'),  Pathology_results, on='ID', how='inner')
    ALL_results = pd.merge(ALL_results, Transformer_results, on='ID', how='inner')

    ALL_results.columns = ['ID', '-0', model_names[0], task, 
                           '-00', model_names[1], '-l', 
                           '-000', model_names[2], '-ll',
                           '-0000', model_names[-1], '-lll']
    ALL_results = normalize_df(ALL_results, not_norm=['ID'], method='minmax')
    Clinic = pd.read_csv('data/clinic_mul.csv')
    # Clinic['ID'] = Clinic['ID'].map(lambda x: f"{x}.nii.gz")
    Clinic = Clinic[[c for c in Clinic.columns if c not in ['label', 'group']]]
    ALL_results = pd.merge(ALL_results, Clinic, on='ID', how='inner')
    ALL_results = ALL_results.dropna(axis=1)
    ALL_results
    
    venn_labels = venn.generate_petal_labels([
                                          set(Habitat_results['ID']),
                                          set(Pathology_results['ID'])])
    fig, ax = venn.venn2(venn_labels, names=['Radiomics', 'Pathology'], figsize=(10, 6))
    fig.savefig(f'img/{subset}_venn_samples.svg', bbox_inches='tight')
    plt.show()
    
    # 计算融合模型
    for mn, columns, model in zip(model_names[-5:-1], 
                                  [list(Clinic.columns[1:]) + ['Habitat'], 
                                   list(Clinic.columns[1:]) + ['Pathology'], 
                                   ['Habitat', 'Pathology'],
                                   list(Clinic.columns[1:]) + ['Habitat', 'Pathology']],
                                  models):

        model = models[mn]
    #     model = SVC(probability=True, random_state=0)
    #     model = RandomForestClassifier(n_estimators=1, max_depth=3, min_samples_split=2, random_state=0)
        data_x = ALL_results[columns]
        data_y = ALL_results[task]
        model.fit(data_x, data_y)
        results = model.predict_proba(data_x)
        results = pd.DataFrame(results, index=ALL_results['ID'], columns=[f'-0', f'{mn}']).reset_index()
        results.to_csv(f'./results/{mn}_{subset}.csv', index=False, header=True)
        ALL_results = pd.merge(ALL_results, results, on='ID', how='inner')
    ALL_results
    
    # 绘制ROC
    gt = [np.array(ALL_results[task]) for d in model_names]
    pred_train = [np.array(ALL_results[d]) for d in model_names]
    okcomp.comp1.draw_roc(gt, pred_train, labels=model_names, title=f"Model AUC")
    plt.savefig(f'img/{subset}_auc.svg')
    plt.show()
    
    # 输出Metrics
    youden = {}
    for mname, y, score in zip(model_names, gt, pred_train):
        # 计算验证集指标
        acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = analysis_pred_binary(y, score)
        ci = f"{ci[0]:.4f} - {ci[1]:.4f}"
        metric.append((mname, acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres, subset))
        youden[mname] = thres
    metric_ = pd.DataFrame(metric, index=None, columns=['Signature', 'Accuracy', 'AUC', '95% CI', 'Sensitivity', 'Specificity',                       
                                                       'PPV', 'NPV', 'Precision', 'Recall', 'F1','Threshold', 'Cohort'])
    display(metric_)
    
    # 计算Delong
    delong = []
    delong_columns = []
    this_delong = []
    plt.figure(figsize=(10, 8))
    cm = np.zeros((len(model_names), len(model_names)))
    for i, mni in enumerate(model_names):
        for j, mnj in enumerate(model_names):
            if i <= j:
                cm[i][j] = np.nan
            else:
                cm[i][j] = delong_roc_test(ALL_results[task], ALL_results[mni], ALL_results[mnj])[0][0]
    cm = pd.DataFrame(cm[1:, :-1], index=model_names[1:], columns=model_names[:-1])
    draw_matrix(cm, annot=True, cmap='jet_r', cbar=True)
    plt.title(f'Cohort {subset} Delong')
    plt.savefig(f'img/delong_each_cohort_{subset}.svg', bbox_inches = 'tight')
    plt.show()
    
    # 计算NRI
    plt.figure(figsize=(10, 8))
    cm = np.zeros((len(model_names), len(model_names)))
    for i, mni in enumerate(model_names):
        for j, mnj in enumerate(model_names):
            cm[i][j] = NRI(ALL_results[mni] > youden[mni], ALL_results[mnj] > youden[mnj], ALL_results[task])
    cm = pd.DataFrame(cm, index=model_names, columns=model_names)
    draw_matrix(cm, annot=True, cmap='jet_r', cbar=True)
    plt.title(f'Cohort {subset} NRI')
    plt.savefig(f'img/all_NRI_each_cohort_{subset}.svg', bbox_inches = 'tight')
    plt.show()
    
    # 计算IDI
    cm = np.zeros((len(model_names), len(model_names)))
    p = np.zeros((len(model_names), len(model_names)))
    for i, mni in enumerate(model_names):
        for j, mnj in enumerate(model_names):
            cm[i][j], p[i][j] = IDI(ALL_results[mni], ALL_results[mnj], ALL_results[task], with_p=True)

    for d, n in zip([cm, p], ['IDI', 'IDI pvalue']):
        plt.figure(figsize=(10, 8))
        d = pd.DataFrame(d, index=model_names, columns=model_names)
        draw_matrix(d, annot=True, cmap='jet_r', cbar=True)
        plt.title(f'Cohort {subset} {n}')
        plt.savefig(f'img/all_{n}_each_cohort_{subset}.svg', bbox_inches = 'tight')
        plt.show()
    
    # DCA
    plot_DCA([ALL_results[model_name] for model_name in model_names], 
             ALL_results[task], title=f'Model for DCA', labels=model_names, remap=False, y_min=-0.15, 
#              EX={'n_estimators':5, 'max_depth': 1, 'random_state':0}, 
             idx_set=[3],
            )
    plt.savefig(f'img/{subset}_dca.svg')
    
    # 校准
    draw_calibration(pred_scores=pred_train, n_bins=5, remap=True,
                     #EX={'n_estimators':5, 'max_depth':4, 'random_state':0}, idx_set=[3],
                     y_test=gt, model_names=model_names)
    plt.savefig(f'img/{subset}_cali.svg')
    
    # HL
    hosmer.append([stats.hosmer_lemeshow_test(y_true, y_pred, bins=19, remap=True, ) 
                   for fn, y_true, y_pred in zip(model_names, gt, pred_train)])
    display(pd.DataFrame(hosmer, columns=model_names))

In [ ]:
import seaborn as sns
from onekey_algo import init_CN

init_CN()
plt.rcParams['font.size'] = get_param_in_cwd('font.size', 20)
plt.figure(figsize=(20, 14))
sns.set_style("ticks")
metric = pd.DataFrame(metric, index=None, columns=['Signature', 'Accuracy', 'AUC', '95% CI', 'Sensitivity', 'Specificity',                       
                                                       'PPV', 'NPV', 'Precision', 'Recall', 'F1','Threshold', 'Cohort'])

def map_name(x):
    if x == 'train':
        return 'Train'
    elif x == 'val':
        return 'Internal Validation'
    else:
        return 'External Test'
metric['Cohort'] = metric['Cohort'].map(lambda x: map_name(x))
sns.barplot(x='Cohort', y='AUC', data=metric, hue='Signature')
plt.ylim(0.5)
plt.savefig('img/auc_comparision.svg', bbox_inches='tight')